In [3]:
import DXYspider_Func as df
import DXYspider_Proxy as dp
import pymongo
import time
import re
import hashlib
import random
import requests
import numpy as np
from tqdm import *
from fake_useragent import UserAgent
import os
import winsound
import win32api
import win32con

In [4]:
client = pymongo.MongoClient('localhost', 27017)
db = client.DXY
collection1 = db.Heart_postId
collection2 = db.Heart_postInfo
collection3 = db.Heart_replyInfo
collection4 = db.proxies




### 获取postId


In [ ]:

for i in range(1, 100):                                 #获取PostId
    proxies = dp.proxy(collection4)
    sTs = df.get_serverTimestamp(proxies)
    for pageNum in range(10*i-9,10*i+1):
        count = 0
        data=df.get_postId(sTs,47,0,pageNum,proxies)
        results=df.parse_postId(data)
        for result in results:  
            #collection1.insert_one(result)
            print(result)
            count += 1
        if pageNum%10 == 0:
            print(f'page{pageNum} , {count} entries')




In [9]:
postIds = collection1.distinct('entityId') 

print(len(postIds))

97177


In [10]:
'''
Note=open('postIds.txt',mode='w')                       #PostId存储
entityIds = collection1.distinct('entityId')
for entityId in entityIds:
    Note.write(str(entityId)+'\n')
Note.close()
'''


### 获取postInfo

In [3]:
postIds = collection2.distinct('postId') 

print(len(postIds))

97150


In [15]:
postIds = df.txt2list('postIds.txt')                #获取PostInfo
count = 0
for i in tqdm(range(len(postIds))):
    proxies = dp.paid_proxy(collection4)
    '''
    timeCurrent=time.time()
    if timeCurrent-timeStart>RandStop:              #随机暂停
        #print(f'{RandStop}s break\n')
        time.sleep(2)
        timeStart=timeCurrent 
        RandStop = random.randint( 60, 120 )                    
    '''
    try:                                            #获取STS
        sTs = df.get_serverTimestamp(proxies)
    except ConnectionResetError:
        print('ConnectionResetError')
        time.sleep(2)
        sTs = df.get_serverTimestamp() 
    except requests.exceptions.RequestException :
        continue            

    try:                                            #拉取postInfo
        data = df.get_postInfo(sTs,postIds[0],proxies)
    except ConnectionResetError:
        time.sleep(2)
        print('ConnectionResetError,retry...\n')    
        continue
    except requests.exceptions.RequestException :
        continue    

    results = df.parse_postInfo(data)               #格式化postInfo

    if data.get('data')!=None:                      #输出至DB54
        for result in results:  
            collection2.insert_one(result) 
        count += 1

    elif data.get('code')=='TD0300000015':          #访问频繁未返回，等待重新拉取 
        win32api.MessageBox(0, "前往验证", "提醒",win32con.MB_ICONWARNING)
        continue
        
    elif data.get('code')=='success':               #若无返回帖子则记录，等待复核 
        g=open('Nodata.txt',mode='a') 
        g.write(str(postIds[0])+'\n')
        g.close() 

    del postIds[0]                                  #移除postId

    if  count!= 0 and count%10 == 0:                #进度更新
        f=open('postIds.txt',mode='w')                             
        for postId in postIds:
            f.write(str(postId)+'\n')
        f.close()

100%|██████████| 3/3 [00:01<00:00,  1.68it/s]


### 获取replyInfo

In [ ]:

'''
Note=open('postIds.txt',mode='w')                       #PostId存储
entityIds = collection3.find(condition)
for entityId in entityIds:
    Note.write(str(entityId.get('postId'))+'\n')
Note.close()
'''

In [11]:
postIds = df.txt2list('postIds.txt')                #获取PostInfo
count = 0
#for i in tqdm(range(len(postIds))):
for i in tqdm(range(len(postIds))):
    condition = {'postId':postIds[0]}
    m = collection3.find_one(condition)
    proxies = dp.paid_proxy(collection4)
    '''
    timeCurrent=time.time()
    if timeCurrent-timeStart>RandStop:              #随机暂停
        #print(f'{RandStop}s break\n')
        time.sleep(2)
        timeStart=timeCurrent 
        RandStop = random.randint( 60, 120 )                    
    '''
    try:                                            #获取STS
        sTs = df.get_serverTimestamp(proxies)
    except ConnectionResetError:
        print('ConnectionResetError')
        time.sleep(2)
        sTs = df.get_serverTimestamp() 
    except requests.exceptions.RequestException :
        continue            

    try:                                            #拉取replyInfo
        data = df.get_replyInfo(postIds[0],sTs,proxies) 
    except ConnectionResetError:
        time.sleep(1)
        print('ConnectionResetError,retry...\n')    
        continue
    except requests.exceptions.RequestException :
        continue    
    
    s = collection3.find_one(condition)
    s['replyList'] = data
    collection3.update_one(condition,{'$set': s}) 

    del postIds[0]                                  #移除postId
    count+=1
    
    if  count!= 0 and count%10 == 0:                #进度更新
        f=open('postIds.txt',mode='w')                             
        for postId in postIds:
            f.write(str(postId)+'\n')
        f.close()

100%|██████████| 12/12 [00:04<00:00,  2.77it/s]


### 获取userInfo

In [18]:
Note=open('postIds.txt',mode='w')                       #PostId存储
entityIds = collection2.distinct('postId')
for entityId in entityIds:
    Note.write(str(entityId)+'\n')
Note.close()

In [18]:
count = 0
for i in tqdm(range(10000,10100)):
    proxies = dp.paid_proxy(collection4)
    '''
    timeCurrent=time.time()
    if timeCurrent-timeStart>RandStop:              #随机暂停
        #print(f'{RandStop}s break\n')
        time.sleep(2)
        timeStart=timeCurrent 
        RandStop = random.randint( 60, 120 )                    
    '''
    try:                                            #获取STS
        sTs = df.get_serverTimestamp(proxies)
    except ConnectionResetError:
        print('ConnectionResetError')
        time.sleep(2)
        sTs = df.get_serverTimestamp() 
    except requests.exceptions.RequestException :
        continue            

    try:                                            #拉取userInfo
        data = df.get_userInfo(sTs,i,proxies)
    except ConnectionResetError:
        time.sleep(2)
        print('ConnectionResetError,retry...\n')    
        continue
    except requests.exceptions.RequestException :
        continue    

    if data.get('data')!=None:                      
        results = df.parse_userInfo(data)               #格式化postInfo
        for result in results:                          #输出至DB54
            collection3.insert_one(result) 
        count += 1

    elif data.get('code')=='TD0300000015':          #访问频繁未返回，等待重新拉取 
        win32api.MessageBox(0, "前往验证", "提醒",win32con.MB_ICONWARNING)
        continue
        
    elif data.get('code')=='success':               #若无返回帖子则记录，等待复核 
        g=open('Nodata.txt',mode='a') 
        g.write(i+'\n')
        g.close() 

100%|██████████| 100/100 [00:21<00:00,  4.74it/s]


db.Heart_postInfo.aggregate([
    {$group: { _id: {userName: '$postId'},count: {$sum: 1},dups: {$addToSet: '$_id'}}},
    {$match: {count: {$gt: 1}}}
]).forEach(function(doc){
    doc.dups.shift();
    db.Heart_postInfo.remove({_id: {$in: doc.dups}});
})